In [34]:
import os
import csv
import shutil
import matplotlib.pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential
from keras import optimizers
from keras.applications import VGG16
from PIL import Image



In [35]:

#create strings for directory access
baseDirectory = r'C:\Users\Administrator\Desktop\Personal-Projects-Blood-Type-Prediction\Dataset'
finalTrainDirectory = r'C:\Users\Administrator\Desktop\Personal-Projects-Blood-Type-Prediction\Dataset\FINALTRAIN'
testDirectory = r'C:\Users\Administrator\Desktop\Personal-Projects-Blood-Type-Prediction\Dataset\TEST'
trainDirectory = r'C:\Users\Administrator\Desktop\Personal-Projects-Blood-Type-Prediction\Dataset\TRAIN'
validationDirectory = r'C:\Users\Administrator\Desktop\Personal-Projects-Blood-Type-Prediction\Dataset\VALIDATION'


In [36]:

#create generators for images
finalTrainGenerator = ImageDataGenerator(rescale = (1/255))
testGenerator = ImageDataGenerator(rescale = (1/255))
trainGenerator = ImageDataGenerator(rescale = (1/255))
validationGenerator = ImageDataGenerator(rescale = (1/255))


#create directory iterators from generators
finalTrainDirectoryIterator = finalTrainGenerator.flow_from_directory(directory = finalTrainDirectory,
                                                                      target_size = (256, 256),
                                                                      class_mode = 'categorical',
                                                                      batch_size = 128)

testDirectoryIterator = testGenerator.flow_from_directory(directory = testDirectory,
                                                                      target_size = (256, 256),
                                                                      class_mode = 'categorical',
                                                                      batch_size = 128)

trainDirectoryIterator = trainGenerator.flow_from_directory(directory = trainDirectory,
                                                                        target_size = (256, 256),
                                                                        class_mode = 'categorical',
                                                                        batch_size = 128)

validationDirectoryIterator = validationGenerator.flow_from_directory(directory = validationDirectory,
                                                                      target_size = (256, 256),
                                                                      class_mode = 'categorical',
                                                                      batch_size = 128)


Found 9957 images belonging to 4 classes.
Found 2487 images belonging to 4 classes.
Found 7957 images belonging to 4 classes.
Found 2000 images belonging to 4 classes.


In [37]:

#create the convolutional base 
convolutionalBase = VGG16( weights = 'imagenet',
                           include_top = False,
                           input_shape = ( 256, 256, 3 ) )

#freeze the convolutional base
convolutionalBase.trainable = False 

#create the sequential model with the VGG16 convolutional base
sequentialModel = Sequential()
sequentialModel.add( convolutionalBase )
sequentialModel.add( Flatten() )
sequentialModel.add( Dropout( 0.5 ) )
sequentialModel.add( Dense( 512, activation = 'relu' ) )
sequentialModel.add( Dense( 4, activation = 'sigmoid' ) )

#compile the model
sequentialModel.compile( optimizer = 'rmsprop',
                       loss = 'categorical_crossentropy',
                       metrics = [ 'acc' ] )

#train the model
sequentialModelHistory = sequentialModel.fit_generator( generator = trainDirectoryIterator,
                                                         steps_per_epoch = 62,
                                                         epochs = 1,
                                                         verbose = True,
                                                         validation_data = validationDirectoryIterator,
                                                         validation_steps = 16 )

#save the model
sequentialModel.save( 'BloodTypeClassifierUsingVGG16Base.h5' )

#retrieve the training history
sequentialModelHistory = sequentialModelHistory.history




Epoch 1/1
62/62 [==============================] - 141s 2s/step - loss: 4.8046 - acc: 0.2494 - val_loss: 4.8535 - val_acc: 0.2500
